In [1]:
import os
import math
import random 
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from Util_def import *
from Util_model import *
from pypfopt import (
    EfficientFrontier,
    risk_models,
    expected_returns,
    objective_functions,
)
import pickle as pkl

import warnings
warnings.filterwarnings('ignore')


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [2]:
ETF_list = [
    'SHV',
    'BND', 'BNDX', 'JNK',
    'VT', 'VEA', 'IEMG',
    'VOO', 'QQQ', 'DIA', 'VGK', 'EWJ', 'MCHI', 'THD', 'VNM', 'INDA',
    'RXI', 'KXI', 'IXC', 'IXG', 'IXJ', 'EXI', 'IXN', 'IXP', 'JXI',
    'ITA', 'ICLN', 'SKYY', 'SMH',
    'REET', 'IGF', 'PDBC', 'GLD'
]

# 5 years data
startDate = dt.datetime(2015, 1, 1)
endDate = dt.datetime(2025, 7, 28)

start_rebalance_year = 2024  # startDate.year + 3

data = getData(ETF_list, startDate, endDate)
data.fillna(method='ffill', inplace=True)
# data.fillna(method='bfill', inplace=True)

# read csv
# data = pd.read_csv('33_ETF_data.csv', index_col='Date', parse_dates=True)
print(data.info())
avg_days = avg_days_per_month(data)

print("=" * 50)
print("Min Date:", data.index.min())
print("Max Date:", data.index.max())
print("Start Rebalance Year:", start_rebalance_year)
print(f"Average number of trading days per month: {avg_days}", "days")
print("=" * 50)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  33 of 33 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2656 entries, 2015-01-02 to 2025-07-25
Data columns (total 33 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SHV     2656 non-null   float64
 1   BND     2656 non-null   float64
 2   BNDX    2656 non-null   float64
 3   JNK     2656 non-null   float64
 4   VT      2656 non-null   float64
 5   VEA     2656 non-null   float64
 6   IEMG    2656 non-null   float64
 7   VOO     2656 non-null   float64
 8   QQQ     2656 non-null   float64
 9   DIA     2656 non-null   float64
 10  VGK     2656 non-null   float64
 11  EWJ     2656 non-null   float64
 12  MCHI    2656 non-null   float64
 13  THD     2656 non-null   float64
 14  VNM     2656 non-null   float64
 15  INDA    2656 non-null   float64
 16  RXI     2656 non-null   float64
 17  KXI     2656 non-null   float64
 18  IXC     2656 non-null   float64
 19  IXG     2656 non-null   float64
 20  IXJ     2656 non-null   float64
 21  EXI     2656 non-nu

In [3]:
###### Portfolio Type ######
long_only = tuple([0,1])
long_short = tuple([-1,1])

port_type = long_only 

###### Adding Constraints ######
# Asset Mapping
asset_map = {
    'SHV': 'Cash_Equivalent',
    
    'BND': 'Fixed_Income',
    'BNDX': 'Fixed_Income',
    'JNK': 'Fixed_Income',

    'VT': 'Equity',
    'VEA': 'Equity',
    'IEMG': 'Equity',

    'VOO': 'Equity',
    'QQQ': 'Equity',
    'DIA': 'Equity',
    'VGK': 'Equity',
    'EWJ': 'Equity',
    'MCHI': 'Equity',
    'THD': 'Equity',
    'VNM': 'Equity',
    'INDA': 'Equity',

    'RXI': 'Equity',
    'KXI': 'Equity',
    'IXC': 'Equity',
    'IXG': 'Equity',
    'IXJ': 'Equity',
    'EXI': 'Equity',
    'IXN': 'Equity',
    'IXP': 'Equity',
    'JXI': 'Equity',

    'ITA': 'Equity',
    'ICLN': 'Equity',
    'SKYY': 'Equity',
    'SMH': 'Equity',

    'REET': 'Alternatives',
    'IGF': 'Alternatives',
    'PDBC': 'Alternatives',
    'GLD': 'Alternatives',
}

### Aggressive Portfolio ###
asset_lower_aggressive = {
    'Cash_Equivalent': 0.0,
    'Fixed_Income': 0.0,
    'Equity': 0.55,
    'Alternatives': 0.0}
asset_upper_aggressive = {
    'Cash_Equivalent': 0.4,
    'Fixed_Income': 0.3,
    'Equity': 0.9,
    'Alternatives': 0.3}

len(ETF_list), len(asset_map)

(33, 33)

# Training

In [4]:
# Directory to save the results
output_dir = 'Results'
model_type = 'tAPE'  # 'Transformer','RNN', 'LSTM', 'BiLSTM', 'GRU', 'BiGRU', '2DCNN', '1DCNN'
pe_type = ''          # 'OriPE', 'Time2Vec', 'ConvSPE', 'SineSPE', 'TemporalPE', 'LearnablePE', 'AbsolutePE', 'tAPE'
pre_post = 'PreNorm'       # 'PostNorm' or 'PreNorm'
n_temp = 1.0
train_type = f'01_1_{model_type}_{pe_type}_{pre_post}_temp_{n_temp}'  # '01_1', '01_2', '01_3', etc.
run_no = 11


# --- เริ่มโค้ดสำหรับบันทึก Excel ---
results_excel_path = f"{output_dir}/Others/{model_type}/{train_type}/01_Results_{model_type}_{pe_type}_{pre_post}_run_{run_no}.xlsx"


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, LayerNormalization, MultiHeadAttention, Embedding, GlobalAveragePooling1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from dateutil.parser import parse
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.models import Model as KModel, Sequential
from tensorflow.keras import layers, Model as KModel
import cvxpy as cp
import cvxopt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Layer


devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)
warnings.filterwarnings('ignore')


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


Model

In [6]:
class Model:
    def __init__(self, max_weight=1, asset_map=None, asset_lower=None, asset_upper=None, port_type=None):
        # self.data = None
        self.model = None
        self.max_weight = max_weight
        self.asset_map = asset_map or {}
        self.asset_lower = asset_lower or {}
        self.asset_upper = asset_upper or {}
        self.port_type = port_type
        self.asset_columns = None

    def _create_constraint_matrices(self, columns):
        """Create constraint matrices for asset‐type bounds."""
        self.asset_columns = columns
        
        asset_types = {}
        for asset in columns:
            t = self.asset_map.get(asset, "Unknown")
            asset_types.setdefault(t, []).append(asset)
        
        mats = []
        lbs = []
        ubs = []
        names = []
        for t, assets in asset_types.items():
            vec = np.zeros(len(columns), dtype=float)
            for a in assets:
                idx = columns.get_loc(a)
                vec[idx] = 1.0
            mats.append(vec)
            lbs.append(self.asset_lower.get(t, 0.0))
            ubs.append(self.asset_upper.get(t, 1.0))
            names.append(t)
        
        self.constraint_matrix = np.vstack(mats)        # shape = (n_types, n_assets)
        self.lower_bounds = np.array(lbs, dtype=float)  # shape = (n_types,)
        self.upper_bounds = np.array(ubs, dtype=float)  # shape = (n_types,)
        self.asset_type_names = names
        
    # Final New QP with two phases - this allows for hard constraints on zero weights
    def _apply_constraints_final(self, weights):
        """
        ใช้ Quadratic Programming เพื่อบังคับ:
        • Σw = 1
        • per-asset cap 0.30 (SHV 0.40)
        • asset_lower / asset_upper
        • ลดการขยับจาก w0 โดยเฉพาะตำแหน่งที่ w0 == 0
        """
        w0 = np.asarray(weights, float).copy()
        n  = w0.size

        # ---------- สร้างขอบรายตัว --------------------------------------------
        ub = np.full(n, 0.30)
        if self.asset_columns is not None and "SHV" in self.asset_columns:
            ub[self.asset_columns.get_loc("SHV")] = 0.40

        lb = np.full(n, self.port_type[0])      # long_only → 0, long/short → -1
        # (ถ้ามีพอร์ตชนิดอื่นปรับได้ตาม self.port_type)

        # ---------- ตัวแปร QP ---------------------------------------------------
        w = cp.Variable(n)

        constraints = [
            cp.sum(w) == 1,
            w >= lb,
            w <= ub
        ]

        # ---------- ข้อจำกัดรายหมวด -------------------------------------------
        if hasattr(self, "constraint_matrix"):
            C = self.constraint_matrix           # shape (n_types, n_assets)
            constraints += [
                C @ w >= self.lower_bounds,
                C @ w <= self.upper_bounds
            ]

        # ---------- Objective: min Σ α_i (w_i - w0_i)^2 ------------------------
        eps = 1e-4
        alpha = 1.0 / (w0 + eps)        # ช่องที่ w0=0 จะถูกลงโทษมาก
        #obj   = cp.Minimize(cp.sum(cp.multiply(alpha, cp.square(w - w0)))) # L2
        obj = cp.Minimize(cp.sum(cp.multiply(alpha, cp.abs(w - w0)))) # L1

        prob = cp.Problem(obj, constraints)

        # เลือก solver ที่รองรับ QP
        try:
            prob.solve(solver=cp.OSQP)  # หรือ ECOS_BB / SCS
        except cp.error.SolverError:
            prob.solve(solver=cp.ECOS)

        # ถ้าแก้ไม่ได้ (infeasible) กลับไปใช้วิธีเดิม
        if w.value is None:
            print("QP infeasible, falling back to greedy method.")
            return super()._apply_constraints(weights)

        return np.asarray(w.value).flatten()
    # ===== END V.2 =====
       
    def _apply_constraints(self, weights: np.ndarray) -> np.ndarray:
        """
        Accept either:
          • 1D array of shape (n_assets,), or
          • 2D array of shape (batch_size, n_assets).
        Returns a NumPy array of the same shape, after applying constraints row‐wise.
        """
        w = np.array(weights, copy=True)
        if w.ndim == 1:
            return self._apply_constraints_final(w)
        elif w.ndim == 2:
            return np.vstack([self._apply_constraints_final(row) for row in w])
        else:
            raise ValueError(f"_apply_constraints got ndim={w.ndim}, expected 1 or 2.")

    def build(self, input_shape, outputs):
        d_model = 512
        num_heads = 8
        ff_dim = 2048
        num_layers = 6
        dropout_rate = 0.3

        # ===== Input Layer =====
        seq_len, feature_dim = input_shape
        inputs = layers.Input(shape=(seq_len, feature_dim))

        # ====== time Absolute Positional Encoding (tAPE) ======
        x = layers.Dense(d_model)(inputs)
        pos_encoding_layer = tAPE(d_model=d_model, max_len=seq_len + 100, dropout=dropout_rate)
        x = pos_encoding_layer(x)

        # ====== Transformer Encoder ======
        for _ in range(num_layers):
            attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
            attn_output = layers.Dropout(dropout_rate)(attn_output)
            out1 = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

            ffn = layers.Dense(ff_dim, activation="relu")(out1)
            ffn = layers.Dense(d_model)(ffn)
            ffn_output = layers.Dropout(dropout_rate)(ffn)
            x = layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

        pooled = layers.GlobalAveragePooling1D()(x)

        #  ===== Output layer =====
        logits = layers.Dense(outputs)(pooled)
        logits = TemperatureSoftmax(temperature=n_temp)(logits)  # Use custom softmax layer
        
        if hasattr(self, "constraint_matrix"):
            print("Applying constraints to output layer... ✅")
            def _tf_apply_constraints(x_tensor):
                y = tf.py_function(self._apply_constraints, [x_tensor], tf.float32)
                y.set_shape([None, outputs])
                return y

            constrained = layers.Lambda(_tf_apply_constraints)(logits)
        else:
            print("No constraints applied to output layer... ❌")
            constrained = logits

        model = KModel(inputs=inputs, outputs=constrained)
                     
        # # ===== Sharpe Ratio Loss Function =====
        # def sharpe_loss(y_true, y_pred):
        #     """
        #     คำนวณ Sharpe Ratio Loss สำหรับ Batch ของข้อมูล
            
        #     Args:
        #         y_true (tf.Tensor): ผลตอบแทนรายวันในอนาคต (future daily returns)
        #                             Shape: (batch_size, forecast_horizon, n_assets)
        #         y_pred (tf.Tensor): น้ำหนักพอร์ตที่โมเดลทำนาย (predicted weights)
        #                             Shape: (batch_size, n_assets)
        #     """
        #     weights_expanded = tf.expand_dims(y_pred, axis=1)
        #     portfolio_daily_returns = tf.reduce_sum(tf.multiply(y_true, weights_expanded), axis=2)

        #     mean_daily_return = tf.reduce_mean(portfolio_daily_returns, axis=1)
        #     std_daily_return = tf.math.reduce_std(portfolio_daily_returns, axis=1)

        #     annualized_return = mean_daily_return * 252.0
        #     annualized_volatility = std_daily_return * tf.sqrt(252.0)
            
        #     risk_free_rate = 0.02
        #     sharpe = (annualized_return - risk_free_rate) / (annualized_volatility + 1e-9)

        #     return -tf.reduce_mean(sharpe)
        
        # optimizer = tf.keras.optimizers.Adam()
        # model.compile(loss=sharpe_loss, optimizer=optimizer)
        
        def sortino_loss(y_true, y_pred):
            """
            คำนวณ Sortino Ratio Loss สำหรับ Batch ของข้อมูล
            
            Args:
                y_true (tf.Tensor): ผลตอบแทนรายวันในอนาคต (future daily returns)
                                    Shape: (batch_size, forecast_horizon, n_assets)
                y_pred (tf.Tensor): น้ำหนักพอร์ตที่โมเดลทำนาย (predicted weights)
                                    Shape: (batch_size, n_assets)
            """
            # ขยายมิติของ weights เพื่อให้คูณกับ y_true ได้
            # Shape: (batch_size, 1, n_assets)
            weights_expanded = tf.expand_dims(y_pred, axis=1)
            
            # คำนวณผลตอบแทนรายวันของพอร์ต
            # Shape: (batch_size, forecast_horizon)
            portfolio_daily_returns = tf.reduce_sum(tf.multiply(y_true, weights_expanded), axis=2)

            # คำนวณผลตอบแทนเฉลี่ยรายวัน
            # Shape: (batch_size,)
            mean_daily_return = tf.reduce_mean(portfolio_daily_returns, axis=1)
            
            # --- ส่วนที่แตกต่างจาก Sharpe Loss ---
            # คัดเฉพาะผลตอบแทนที่ต่ำกว่าเป้าหมาย (ในที่นี้คือ 0) เพื่อคำนวณ Downside Risk
            # Shape: (batch_size, forecast_horizon)
            negative_returns = tf.minimum(portfolio_daily_returns, 0.0)
            
            # คำนวณ Downside Deviation (ส่วนเบี่ยงเบนมาตรฐานของผลตอบแทนที่ติดลบ)
            # Shape: (batch_size,)
            downside_std = tf.sqrt(tf.reduce_mean(tf.square(negative_returns), axis=1))
            # --- สิ้นสุดส่วนที่แตกต่าง ---

            # แปลงเป็นหน่วยต่อปี
            annualized_return = mean_daily_return * 252.0
            annualized_downside_volatility = downside_std * tf.sqrt(252.0)
            
            # ตั้งค่า Risk-Free Rate
            risk_free_rate = 0.02
            
            # คำนวณ Sortino Ratio
            # เพิ่ม 1e-9 เพื่อป้องกันการหารด้วยศูนย์
            sortino = (annualized_return - risk_free_rate) / (annualized_downside_volatility + 1e-9)

            # เราต้องการ "Maximize" Sortino Ratio ดังนั้น Loss คือค่าติดลบของมัน
            return -tf.reduce_mean(sortino)
        optimizer = tf.keras.optimizers.Adam()
        model.compile(loss=sortino_loss, optimizer=optimizer)
        return model

    def calc_wgts(self, lkbk: int, ep: int, data_prices: pd.DataFrame, features: pd.DataFrame, 
                  forecast_horizon: int, patience=10, rebalance_date=None):
        
        if not hasattr(self, "constraint_matrix"):
            self._create_constraint_matrices(data_prices.columns)

        X_train, y_train = create_direct_forecast_dataset(
            features, 
            data_prices, 
            sequence_length=lkbk, 
            forecast_horizon=forecast_horizon
        )
        if len(X_train) == 0:
            raise ValueError("Could not create any training samples. Check data length, lookback, and horizon.")

        input_shape_for_build = (X_train.shape[1], X_train.shape[2])
        self.model = self.build(input_shape_for_build, len(data_prices.columns))
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

        history = self.model.fit(X_train, y_train, epochs=ep, batch_size=64, 
                       shuffle=False,
                       validation_split=0.2, 
                       callbacks=[early_stopping], verbose=1)


        latest_features = features.iloc[-lkbk:].values
        
        predict_data = latest_features[np.newaxis, :, :]

        weights = self.model.predict(predict_data)[0]
        return weights

In [7]:
# Direct Multi-step Forecasting with Quarterly Walk-Forward Analysis
def quarterly_walk_forward(df, lookback, n_epochs, features, asset_map=None, 
                           asset_lower=None, asset_upper=None, port_type=(0, 1), 
                           start_year=2020, trading_days_per_quarter=63, 
                           min_train_periods=252):
    
    original_index = df.index
    df_reset = df.reset_index(drop=True)
    features_reset = features.reset_index(drop=True)
    
    rebalance_dates = get_rebalance_dates(df, start_year)
    
    all_rets, weights_list, rebalance_info = [], [], []
    
    model = Model(
        max_weight=1, asset_map=asset_map, asset_lower=asset_lower,
        asset_upper=asset_upper, port_type=port_type
    )
    
    print(f"\n=== Starting Quarterly Rebalancing with Direct Multi-step Forecast ===")
    
    for i, rebalance_date in enumerate(rebalance_dates):
        print(f"\n--- Rebalancing {i+1}/{len(rebalance_dates)} | Date: {rebalance_date.strftime('%Y-%m-%d')} ---")
        
        try:
            train_end_pos = original_index.get_indexer([rebalance_date], method='pad')[0]
            if train_end_pos < min_train_periods:
                print("Not enough historical data, skipping...")
                continue
        except Exception as e:
            print(f"Cannot find position for {rebalance_date}, skipping... Error: {e}")
            continue
        
        train_start_pos = 0 # Use all available history up to rebalance date
        
        print(f"Training period: {original_index[train_start_pos].strftime('%Y-%m-%d')} to {original_index[train_end_pos-1].strftime('%Y-%m-%d')}")
        
        quarter_end = get_quarter_end_date(rebalance_date, original_index, trading_days_per_quarter)
        
        try:
            test_end_pos = original_index.get_loc(quarter_end)
        except Exception:
            print(f"Cannot find end date for quarter, skipping...")
            continue
            
        # ใช้ df_reset และ features_reset สำหรับการ slicing ด้วย train_end_pos
        train_data_prices = df_reset.iloc[train_start_pos:train_end_pos].copy()
        train_features = features_reset.iloc[train_start_pos:train_end_pos].copy()

        scaler = StandardScaler()
        train_features_scaled = pd.DataFrame(scaler.fit_transform(train_features), index=train_features.index, columns=train_features.columns)
        

        test_data = df_reset.iloc[train_end_pos-1:test_end_pos].copy()
        test_returns = test_data.pct_change().fillna(0).iloc[1:]

        print(f"Training data shape: {train_data_prices.shape}")
        
        if len(test_returns) == 0:
            print("No test returns available, stopping.")
            break

        try:
            K.clear_session()
            
            # ส่ง forecast_horizon=trading_days_per_quarter เข้าไปใน calc_wgts
            asset_weights = model.calc_wgts(
                lkbk=lookback, 
                ep=n_epochs, 
                data_prices=train_data_prices, 
                features=train_features_scaled, # train_features, train_features_scaled
                forecast_horizon=trading_days_per_quarter, # <--- เพิ่ม parameter นี้
                patience=10, 
                rebalance_date=rebalance_date.strftime('%Y-%m-%d')
            )
            
            # -- ส่วนของการแสดงผลและเก็บข้อมูล (เหมือนเดิม) --
            print(f"\n{'=' * 30}")
            # total weight should be 1 status
            total_weight = np.sum(asset_weights).round(4)
            if total_weight == 1:
                print(f"Total weight: {total_weight:.4f} ✅")
            else:
                print(f"Total weight: {total_weight:.4f} ❌ (should be 1.0)")
            
            weights_non_zero_count = np.count_nonzero(asset_weights)
            print(f"Selected ETF count: {weights_non_zero_count}")
            # if weights_non_zero_count < 10 or weights_non_zero_count > 15:
            #     print(f"Warning: weights have {weights_non_zero_count} ❌")
            #     os.system(f'say "Warning: Selected ETF have {weights_non_zero_count}"')
            # if weights_non_zero_count >= 10 and weights_non_zero_count <= 15:
            #     print(f"Selected ETF: {weights_non_zero_count} ✅")

            # Print allocation summary
            if hasattr(model, 'constraint_matrix') and hasattr(model, 'asset_type_names'):
                type_weights = np.dot(model.constraint_matrix, asset_weights)
                print("Asset Type Allocation:")
                for j, (asset_type, weight) in enumerate(zip(model.asset_type_names, type_weights)):
                    lower = model.asset_lower.get(asset_type, 0.0)
                    upper = model.asset_upper.get(asset_type, 1.0)
                    # status
                    if lower <= weight.round(4) <= upper:
                        status = "✅" #Within Range
                    else:
                        status = "❌" #Out of Range
                    print(f"  {asset_type}: {weight:.3f} ({weight*100:.2f}%) [Range: {lower:.2f}-{upper:.2f}] {status}")
            
            # Calculate out-of-sample returns
            if len(test_returns) > 0:
                # Vectorized return calculation
                oos_returns = (test_returns.values * asset_weights).sum(axis=1)
                oos_returns = pd.Series(oos_returns, index=test_returns.index)
                
                all_rets.append(oos_returns)
                weights_list.append(asset_weights)
                rebalance_info.append({
                    'rebalance_date': rebalance_date,
                    'quarter_end': quarter_end,
                    'quarter_return': oos_returns.sum(),
                    'quarter_days': len(oos_returns)
                })
                
                print(f"Quarter return: {oos_returns.sum():.4f} ({oos_returns.sum()*100:.2f}%)")
            else:
                print("No test returns available ❌❌❌")
            
            print(f"\n{'=' * 30}")
                
        except Exception as e:
            print(f"Error in rebalancing: {str(e)} ❌❌❌")
            continue
    
    # Combine results efficiently
    if all_rets:
        pnl = pd.concat(all_rets, ignore_index=False)
        weights_df = pd.DataFrame(weights_list, columns=df.columns)
        rebalance_summary = pd.DataFrame(rebalance_info)
        
        print(f"\n=== Rebalancing Summary ===")
        print(f"Total quarters processed: {len(rebalance_info)}")
        print(f"Total return periods: {len(pnl)}")
        print(f"Average quarterly return: {rebalance_summary['quarter_return'].mean():.4f}")
        print(f"Quarterly return std: {rebalance_summary['quarter_return'].std():.4f}")
        
        return pnl, weights_df, rebalance_summary
    else:
        print("No successful rebalancing periods ❌❌❌")
        return None, None, None


In [8]:
data_train = data.copy()
features = data_train.pct_change().fillna(0)  # Calculate percentage change for features

# Reset states generated by Keras
K.clear_session()

set_seed(1)

n_lookback = avg_days * 3 # Sequence: โค้ดจะสร้าง Input โดยสำหรับ ทุกๆ วัน ในชุดข้อมูลเทรน มันจะ "มองย้อนกลับไป" เป็นจำนวน n_lookback วัน (เช่น 21 วันทำการ หรือประมาณ 1 เดือน)
n_trading_days_per_quarter = avg_days * 3  # 21 days/month * 3 months = 63 days
n_min_train_periods = avg_days * 12 * 3  # 3 years of training data = 252*3 days = 756 days

print(f"Lookback period: {n_lookback} days")
print(f"Trading days per quarter: {n_trading_days_per_quarter} days")
print(f"Minimum training periods: {n_min_train_periods} days")

# Run quarterly rebalancing with constraints
pnl, model_weights, rebalance_summary = quarterly_walk_forward(
    data_train, 
    lookback=n_lookback, 
    n_epochs=100, 
    features=features.fillna(0),
    asset_map=asset_map, 
    asset_lower=asset_lower_aggressive,
    asset_upper=asset_upper_aggressive,
    port_type=long_only,
    start_year=start_rebalance_year,
    trading_days_per_quarter=n_trading_days_per_quarter,  
    min_train_periods=n_min_train_periods,
)

Lookback period: 63 days
Trading days per quarter: 63 days
Minimum training periods: 756 days
Rebalance Dates: ['2024-01-02', '2024-04-01', '2024-07-01', '2024-10-01', '2025-01-02', '2025-04-01', '2025-07-01']

=== Starting Quarterly Rebalancing with Direct Multi-step Forecast ===

--- Rebalancing 1/7 | Date: 2024-01-02 ---
Training period: 2015-01-02 to 2023-12-29
Training data shape: (2264, 33)


2025-07-29 15:13:06.537367: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-07-29 15:13:06.537411: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-29 15:13:06.537416: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-07-29 15:13:06.537432: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-29 15:13:06.537441: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Applying constraints to output layer... ✅
Epoch 1/100


2025-07-29 15:13:09.052444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


27/27 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - loss: -1.2311 - val_loss: -0.3287
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - loss: -1.2291 - val_loss: -0.3287
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - loss: -1.2351 - val_loss: -0.3287
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - loss: -1.2300 - val_loss: -0.3287
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: -1.2335 - val_loss: -0.3287
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - loss: -1.2311 - val_loss: -0.3287
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: -1.2288 - val_loss: -0.3287
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - loss: -1.2307 - val_loss: -0.3287
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 582s 22s/step - loss: -1.2295 - val_loss: -0.3287
Epoch 10/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 546s 21s/step - loss: -1.2316 - val_loss: -0.3287
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 564s 22s/step - loss: -1.2319 - val_loss: -0.3287
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step

Total w

In [9]:
rebalance_dates =  get_rebalance_dates(data, start_year=start_rebalance_year)

# add rebalance_dates to weights_df index
weights_df = model_weights.copy()
weights_df.index = rebalance_dates[:len(weights_df)]
print("\n%Weights DataFrame with Rebalance Dates:")
display(weights_df.multiply(100))

Rebalance Dates: ['2024-01-02', '2024-04-01', '2024-07-01', '2024-10-01', '2025-01-02', '2025-04-01', '2025-07-01']

%Weights DataFrame with Rebalance Dates:


,SHV,BND,BNDX,JNK,VT,VEA,IEMG,VOO,QQQ,DIA,...,IXP,JXI,ITA,ICLN,SKYY,SMH,REET,IGF,PDBC,GLD
2024-01-02,2.878711,1.040354,4.220895,1.703373,4.207602,0.322607,1.484413,2.386879,5.458936,2.796318,...,1.028720,1.248358,6.428210,0.801358,2.986769,3.923821,3.793630,1.809879,2.630692,1.802760
2024-04-01,7.469411,3.069885,2.496007,2.236206,3.679330,0.715805,1.323406,11.098472,3.445110,3.497412,...,2.769415,1.509862,0.777391,1.546371,1.460764,4.646584,1.926754,1.391402,5.386458,0.505223
2024-07-01,2.180279,1.934277,1.685372,1.443530,1.344622,10.325641,4.744483,2.551852,2.132705,0.928922,...,2.258216,10.818873,1.131271,6.680762,0.328898,6.662828,1.832469,2.822638,2.607607,7.176110
2024-10-01,1.022876,1.199011,1.629916,6.828358,2.038009,1.338544,5.987009,2.191418,2.349823,2.073394,...,2.180124,10.167146,1.460163,1.345871,7.643420,1.646034,1.286687,4.074359,6.235471,0.830651
2025-01-02,1.619688,1.893982,4.107817,0.881817,7.675104,1.307544,2.945342,7.180949,1.277338,1.123502,...,2.763629,11.704146,1.982105,2.772083,5.107693,2.218545,4.632481,3.372675,1.010813,2.381694
2025-04-01,1.090854,2.323193,3.257939,0.991733,2.707372,5.786789,0.617854,1.532282,4.793530,6.320457,...,3.080289,1.177984,0.848247,2.173826,3.841621,8.787041,0.628631,1.953465,0.953303,5.362763
2025-07-01,2.644873,3.129056,3.937392,5.922777,0.531335,1.958222,0.699049,0.882673,6.010715,3.452162,...,2.863673,1.001550,0.772884,3.222807,1.790421,3.099973,5.457877,4.323691,2.661546,3.438329


In [10]:
# --- Run the Check and Print Results ---
violations_found = check_portfolio_constraints(
    weights_df, 
    asset_map, 
    asset_lower_aggressive, 
    asset_upper_aggressive
)

if not violations_found:
    print("✅ All portfolio weights satisfy the constraints.")
else:
    print("❌ Constraint violations were found:")
    for date, messages in violations_found.items():
        print(f"\nOn {date}:")
        for msg in messages:
            print(f"  - {msg}")

✅ All portfolio weights satisfy the constraints.


In [11]:
# weights_df.to_csv(f'{output_dir}/{train_type}/raw_weights_df_run{run_no}.csv', index=True)
save_dataframe_to_new_sheet(weights_df, results_excel_path, 'Raw Weights')

DataFrame saved to sheet 'Raw Weights' in new file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/01_Results_tAPE__PreNorm_run_11.xlsx ✨


In [12]:
os.system('say "Model training has finished"')

0

# Compared

### 1. Model weights

In [13]:
model_weights_df = weights_df.copy()
model_weights_df = model_weights_df.round(4)
model_weights_df = model_weights_df.abs()
save_dataframe_to_new_sheet(model_weights_df, results_excel_path, 'Model Weights')
# model_weights_df

DataFrame saved to sheet 'Model Weights' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/01_Results_tAPE__PreNorm_run_11.xlsx 📄


In [14]:
# check number of weights > 0 in each row
non_zero_weights_count = (model_weights_df > 0).sum(axis=1)
print("\nNumber of non-zero weights in each row:")
print(non_zero_weights_count)


Number of non-zero weights in each row:
2024-01-02    33
2024-04-01    33
2024-07-01    33
2024-10-01    33
2025-01-02    33
2025-04-01    33
2025-07-01    33
dtype: int64


In [15]:
# check total weight in each row to ensure it sums to 1
total_weights = model_weights_df.sum(axis=1).round(2)
print("\nTotal weights in each row (should be 1):")
print(total_weights)


Total weights in each row (should be 1):
2024-01-02    1.0
2024-04-01    1.0
2024-07-01    1.0
2024-10-01    1.0
2025-01-02    1.0
2025-04-01    1.0
2025-07-01    1.0
dtype: float32


### 2. Traditional mean-variance optimization

In [16]:
risk_free = 0.02  # Example risk-free rate
mvo_weights_df = mvo_quarterly_rebalancing(data, asset_map, 
                                           asset_lower_aggressive, asset_upper_aggressive, 
                                           port_type, start_year=start_rebalance_year,
                                           trading_days_per_quarter=n_trading_days_per_quarter,
                                           min_train_periods=n_min_train_periods,
                                           risk_free_rate=risk_free)

mvo_weights_df = mvo_weights_df.round(4)
save_dataframe_to_new_sheet(mvo_weights_df, results_excel_path, 'MVO Weights')
# mvo_weights_df

Rebalance Dates: ['2024-01-02', '2024-04-01', '2024-07-01', '2024-10-01', '2025-01-02', '2025-04-01', '2025-07-01']

--- Rebalancing 1/7 ---
Rebalance Date: 2024-01-02
Training period: 2020-12-28 to 2023-12-29
Training days: 757

Optimal Weights:
Alternatives: 0.1500
Cash_Equivalent: 0.3000
Equity: 0.5500
Fixed_Income: 0.0000

--- Rebalancing 2/7 ---
Rebalance Date: 2024-04-01
Training period: 2021-03-26 to 2024-03-28
Training days: 757

Optimal Weights:
Alternatives: 0.1974
Cash_Equivalent: 0.2526
Equity: 0.5500
Fixed_Income: 0.0000

--- Rebalancing 3/7 ---
Rebalance Date: 2024-07-01
Training period: 2021-06-25 to 2024-06-28
Training days: 757

Optimal Weights:
Alternatives: 0.2414
Cash_Equivalent: 0.2086
Equity: 0.5500
Fixed_Income: 0.0000

--- Rebalancing 4/7 ---
Rebalance Date: 2024-10-01
Training period: 2021-09-27 to 2024-09-30
Training days: 757

Optimal Weights:
Alternatives: 0.3000
Cash_Equivalent: 0.1500
Equity: 0.5500
Fixed_Income: 0.0000

--- Rebalancing 5/7 ---
Rebalance D

In [17]:
# --- Run the Check and Print Results ---
violations_found = check_portfolio_constraints(
    mvo_weights_df, 
    asset_map, 
    asset_lower_aggressive, 
    asset_upper_aggressive
)

if not violations_found:
    print("✅ All portfolio weights satisfy the constraints.")
else:
    print("❌ Constraint violations were found:")
    for date, messages in violations_found.items():
        print(f"\nOn {date}:")
        for msg in messages:
            print(f"  - {msg}")

# check number of weights > 0 in each row
non_zero_weights_count = (mvo_weights_df > 0).sum(axis=1)
print("\nNumber of non-zero weights in each row:")
print(non_zero_weights_count)

# check total weight in each row to ensure it sums to 1
total_weights = mvo_weights_df.sum(axis=1).round(2)
print("\nTotal weights in each row (should be 1):")
print(total_weights)

✅ All portfolio weights satisfy the constraints.

Number of non-zero weights in each row:
2024-01-02    6
2024-04-01    6
2024-07-01    5
2024-10-01    7
2025-01-02    5
2025-04-01    5
2025-07-01    7
dtype: int64

Total weights in each row (should be 1):
2024-01-02    1.0
2024-04-01    1.0
2024-07-01    1.0
2024-10-01    1.0
2025-01-02    1.0
2025-04-01    1.0
2025-07-01    1.0
dtype: float64


### 3. Equal weights

In [18]:
# Run equal weight portfolio
equal_weights_df = equal_weight_portfolio(data, rebalance_dates)
equal_weights_df = equal_weights_df.round(4)
save_dataframe_to_new_sheet(equal_weights_df, results_excel_path, 'Equal Weights')
# equal_weights_df

DataFrame saved to sheet 'Equal Weights' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/01_Results_tAPE__PreNorm_run_11.xlsx 📄


### 4. Benchmark

In [19]:
benchmark_df = pd.DataFrame(
   index=rebalance_dates,
   columns=data.columns,
   data=0.0
)
num_cash = sum(1 for v in asset_map.values() if v == 'Cash_Equivalent')
num_fixed_income = sum(1 for v in asset_map.values() if v == 'Fixed_Income')
num_alternatives = sum(1 for v in asset_map.values() if v == 'Alternatives')
print(f"Number of Cash Equivalents: {num_cash}")
print(f"Number of Fixed Income: {num_fixed_income}")
print(f"Number of Alternatives: {num_alternatives}")

benchmark_df['SHV'] = 0.05 / num_cash
benchmark_df[['BND', 'BNDX', 'JNK']] = 0.05 / num_fixed_income
benchmark_df['VT'] = 0.75
benchmark_df[['REET', 'IGF', 'PDBC', 'GLD']] = 0.15 / num_alternatives
save_dataframe_to_new_sheet(benchmark_df, results_excel_path, 'Beanchmark Weights')
# benchmark_df

Number of Cash Equivalents: 1
Number of Fixed Income: 3
Number of Alternatives: 4
DataFrame saved to sheet 'Beanchmark Weights' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/01_Results_tAPE__PreNorm_run_11.xlsx 📄


### 4. Compared

In [20]:
# Main analysis
start_date = rebalance_dates[0]  # First rebalance date

# Calculate portfolio returns for each strategy
portfolios = {
    'Model Portfolio': model_weights_df,
    'MVO Portfolio': mvo_weights_df,
    'Equal Weight': equal_weights_df,
    'Benchmark': benchmark_df
}

portfolio_returns = {}
for name, weights in portfolios.items():
    returns = calculate_portfolio_returns(data, weights, rebalance_dates, start_date)
    portfolio_returns[name] = returns

# Calculate performance metrics
performance_metrics = {}
benchmark_returns = portfolio_returns['Benchmark']

for name, returns in portfolio_returns.items():
    if name == 'Benchmark':
        metrics = calculate_performance_metrics(returns, returns)  # Self as benchmark
    else:
        metrics = calculate_performance_metrics(returns, benchmark_returns)
    performance_metrics[name] = metrics

# Create performance comparison DataFrame
performance_df = pd.DataFrame(performance_metrics).T
print("Portfolio Performance Comparison:")
print("=" * 50)
# performance_df.to_csv(f'{output_dir}/{train_type}/performance_comparison_run{run_no}.csv')
save_dataframe_to_new_sheet(performance_df.T, results_excel_path, 'Performance Comparison')
performance_df.round(4).T

Portfolio Performance Comparison:
DataFrame saved to sheet 'Performance Comparison' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/01_Results_tAPE__PreNorm_run_11.xlsx 📄


,Model Portfolio,MVO Portfolio,Equal Weight,Benchmark
Total Return (%),30.6087,37.2238,28.1150,29.5693
Annualized Return (%),18.7281,22.5596,17.2658,18.1198
Volatility (%),13.1744,11.7621,12.3777,12.8429
Sharpe Ratio,1.2169,1.6188,1.1870,1.2052
Max Drawdown (%),-12.8309,-8.2265,-12.3852,-13.6563
Max Drawdown Duration (days),57.0000,55.0000,79.0000,60.0000
Sortino Ratio,1.6314,1.9872,1.5562,1.5334
Treynor Ratio,0.1584,0.2497,0.1535,0.1544
Jensen's Alpha (%),0.3616,7.2366,-0.1212,-0.0396
Beta,1.0124,0.7626,0.9571,1.0026


In [21]:
# Plotting
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Cumulative Returns', 'Maximum Drawdown'),
    vertical_spacing=0.12,
    row_heights=[0.7, 0.3]
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Plot cumulative returns
for i, (name, returns) in enumerate(portfolio_returns.items()):
    #if name != 'Benchmark':
    cumulative_returns = (1 + returns).cumprod()
    fig.add_trace(
        go.Scatter(
            x=cumulative_returns.index,
            y=cumulative_returns.values,
            mode='lines',
            name=name,
            line=dict(color=colors[i], width=2),
            showlegend=True
        ),
        row=1, col=1
    )

# Plot drawdowns
for i, (name, returns) in enumerate(portfolio_returns.items()):
#if name != 'Benchmark':
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdown = (cumulative - rolling_max) / rolling_max * 100
    
    fig.add_trace(
        go.Scatter(
            x=drawdown.index,
            y=drawdown.values,
            mode='lines',
            name=name,
            line=dict(color=colors[i], width=2),
            showlegend=False
        ),
        row=2, col=1
    )

# Update layout
fig.update_layout(
    title='Portfolio Performance Comparison',
    height=800,
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Return", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

fig.show()

# Save the plot to a file
fig.write_html(f'{output_dir}/Others/{model_type}/{train_type}/portfolio_performance_{model_type}_run_{run_no}.html')
# save image to file
fig.write_image(f'{output_dir}/Others/{model_type}/{train_type}/portfolio_performance_{model_type}_run_{run_no}.png')

In [22]:
# Summary statistics
print("\nKey Performance Highlights:")
print("=" * 30)

best_return = performance_df['Annualized Return (%)'].idxmax()
print(f"Best Annualized Return: {best_return} ({performance_df.loc[best_return, 'Annualized Return (%)']:.2f}%)")

best_sharpe = performance_df['Sharpe Ratio'].idxmax()
print(f"Best Sharpe Ratio: {best_sharpe} ({performance_df.loc[best_sharpe, 'Sharpe Ratio']:.3f})")

best_sortino = performance_df['Sortino Ratio'].idxmax()
print(f"Best Sortino Ratio: {best_sortino} ({performance_df.loc[best_sortino, 'Sortino Ratio']:.3f})")

lowest_dd = performance_df['Max Drawdown (%)'].idxmax()  # Most negative (lowest)
print(f"Lowest Max Drawdown: {lowest_dd} ({performance_df.loc[lowest_dd, 'Max Drawdown (%)']:.2f}%)")

lowest_vol = performance_df['Volatility (%)'].idxmin()
print(f"Lowest Volatility: {lowest_vol} ({performance_df.loc[lowest_vol, 'Volatility (%)']:.2f}%)")


Key Performance Highlights:
Best Annualized Return: MVO Portfolio (22.56%)
Best Sharpe Ratio: MVO Portfolio (1.619)
Best Sortino Ratio: MVO Portfolio (1.987)
Lowest Max Drawdown: MVO Portfolio (-8.23%)
Lowest Volatility: MVO Portfolio (11.76%)


In [23]:
# os.system('say "Model comparison has finished"')

# Quarterly Comparison

In [24]:
# Get quarterly periods
quarterly_periods = get_quarterly_periods(rebalance_dates, data)
quarterly_periods

# Portfolio definitions
portfolios = {
    'Model Portfolio': model_weights_df,
    'MVO Portfolio': mvo_weights_df,
    'Equal Weight': equal_weights_df,
    'Benchmark': benchmark_df
}

# Calculate quarterly performance for each portfolio
quarterly_results = {}
weights_df_2 = weights_df.copy
for portfolio_name, weights_df_2 in portfolios.items():
    quarterly_results[portfolio_name] = {}
    
    for period in quarterly_periods:
        quarter = period['quarter']
        returns = calculate_quarterly_portfolio_returns(data, weights_df_2, period)
        metrics = calculate_quarterly_metrics(returns)
        quarterly_results[portfolio_name][quarter] = metrics

# Create comprehensive results DataFrame
all_metrics = ['Total Return (%)', 'Annualized Return (%)',
               'Volatility (%)', 'Max Drawdown (%)', 
               'Max Drawdown Duration (days)', 'Sharpe Ratio', 'Sortino Ratio']
quarterly_comparison = {}

for metric in all_metrics:
    quarterly_comparison[metric] = pd.DataFrame({
        portfolio: {quarter: quarterly_results[portfolio][quarter][metric] 
                   for quarter in quarterly_results[portfolio]}
        for portfolio in portfolios.keys()
    })

# Display results
for metric in all_metrics:
    print(f"\n{metric}")
    print("-" * 40)
    print(quarterly_comparison[metric].round(4))
    # quarterly_comparison[metric].to_csv(f'{output_dir}/{train_type}/quarterly_{metric.lower().replace(" ", "_")}_run{run_no}.csv', index=True)



Total Return (%)
----------------------------------------
         Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
Q1 2024           6.7584         7.2973        6.0145     7.1044
Q2 2024           2.0421         3.2837        1.6278     2.4182
Q3 2024           8.0386         1.7535        7.6447     6.6061
Q4 2024          -2.4791        -2.3762       -2.9941    -0.8207
Q1 2025           0.5110         8.2697        1.6246     0.5494
Q2 2025          10.6952        10.1618        8.8408     9.0000
Q3 2025           1.9763         2.6683        2.7380     2.1102

Annualized Return (%)
----------------------------------------
         Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
Q1 2024          31.6099        34.4229       27.8008    33.4108
Q2 2024           8.5637        14.0335        6.7832    10.1992
Q3 2024          36.2436         7.2007       34.2674    29.1603
Q4 2024          -9.5536        -9.1713      -11.4493    -3.2426
Q1 2025           2.2008        

In [25]:
# --- เริ่มโค้ดสำหรับบันทึก Excel ---
qoq_excel_path = f"{output_dir}/Others/{model_type}/{train_type}/02_Quarterly_{model_type}_{pe_type}_{pre_post}_run_{run_no}.xlsx"

for metric in all_metrics:
    sheet_name = metric.replace(" ", "_")
    df_to_save = quarterly_comparison[metric].round(4)
    save_dataframe_to_new_sheet(df_to_save, qoq_excel_path, sheet_name)


DataFrame saved to sheet 'Total_Return_(%)' in new file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx ✨
DataFrame saved to sheet 'Annualized_Return_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Volatility_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Max_Drawdown_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Max_Drawdown_Duration_(days)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Sharpe_Ratio' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/02_Quarterly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Sortino_Ratio' in existing file: Results/Others/tAPE/01_1_tAPE

In [26]:
# Plotting
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Quarterly Returns (%)', 'Quarterly Volatility (%)', 
                   'Quarterly Sharpe Ratio', 'Quarterly Max Drawdown (%)'),
    vertical_spacing=0.2,
    horizontal_spacing=0.2
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
quarters = list(quarterly_comparison['Total Return (%)'].index)

# Plot quarterly returns
for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=quarters,
            y=quarterly_comparison['Total Return (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=True
        ),
        row=1, col=1
    )

for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=quarters,
            y=quarterly_comparison['Volatility (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=1, col=2
    )

for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=quarters,
            y=quarterly_comparison['Sharpe Ratio'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=2, col=1
    )

for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=quarters,
            y=quarterly_comparison['Max Drawdown (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=2, col=2
    )

fig.update_layout(
    title='Quarterly Portfolio Performance Comparison',
    height=700,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
fig.update_yaxes(title_text="Return (%)", row=1, col=1)
fig.update_yaxes(title_text="Volatility (%)", row=1, col=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
fig.update_yaxes(title_text="Max Drawdown (%)", row=2, col=2)

fig.show()

# Summary Statistics by Quarter
print("\n" + "="*60)
print("QUARTERLY PERFORMANCE SUMMARY")
print("="*60)

for quarter in quarters:
    print(f"\n{quarter}:")
    print("-" * 20)
    
    # Best performing portfolio this quarter
    quarter_returns = quarterly_comparison['Total Return (%)'].loc[quarter]
    best_return = quarter_returns.idxmax()
    print(f"Best Return: {best_return} ({quarter_returns[best_return]:.2f}%)")
    
    # Best Sharpe ratio this quarter
    quarter_sharpe = quarterly_comparison['Sharpe Ratio'].loc[quarter].dropna()
    if not quarter_sharpe.empty:
        best_sharpe = quarter_sharpe.idxmax()
        print(f"Best Sharpe: {best_sharpe} ({quarter_sharpe[best_sharpe]:.3f})")
    
    # Lowest volatility this quarter
    quarter_vol = quarterly_comparison['Volatility (%)'].loc[quarter].dropna()
    if not quarter_vol.empty:
        lowest_vol = quarter_vol.idxmin()
        print(f"Lowest Vol: {lowest_vol} ({quarter_vol[lowest_vol]:.2f}%)")

# Ranking Analysis
print("\n" + "="*60)
print("PORTFOLIO RANKINGS BY QUARTER")
print("="*60)

ranking_df = pd.DataFrame(index=quarters, columns=portfolios.keys())

for quarter in quarters:
    # Rank by quarterly returns (1 = best)
    quarter_returns = quarterly_comparison['Total Return (%)'].loc[quarter]
    ranks = quarter_returns.rank(ascending=False, method='min')
    ranking_df.loc[quarter] = ranks

print("\nRanking by Quarterly Returns (1=Best, 4=Worst):")
print(ranking_df.astype(int))

# Average ranking
avg_ranking = ranking_df.mean().sort_values()
print(f"\nAverage Ranking Across All Quarters:")
print("-" * 40)
for portfolio, avg_rank in avg_ranking.items():
    print(f"{portfolio}: {avg_rank:.2f}")

# Win rate analysis
print(f"\nQuarterly Win Rate (% of quarters ranked #1):")
print("-" * 45)
for portfolio in portfolios.keys():
    win_rate = (ranking_df[portfolio] == 1).sum() / len(quarters) * 100
    print(f"{portfolio}: {win_rate:.1f}%")

# Save the plot to a file
fig.write_html(f'{output_dir}/Others/{model_type}/{train_type}/QoQ_performance_{model_type}_run_{run_no}.html')
fig.write_image(f'{output_dir}/Others/{model_type}/{train_type}/QoQ_performance_{model_type}_run_{run_no}.png')


QUARTERLY PERFORMANCE SUMMARY

Q1 2024:
--------------------
Best Return: MVO Portfolio (7.30%)
Best Sharpe: MVO Portfolio (4.354)
Lowest Vol: MVO Portfolio (6.39%)

Q2 2024:
--------------------
Best Return: MVO Portfolio (3.28%)
Best Sharpe: MVO Portfolio (1.278)
Lowest Vol: Equal Weight (8.83%)

Q3 2024:
--------------------
Best Return: Model Portfolio (8.04%)
Best Sharpe: Equal Weight (2.287)
Lowest Vol: Equal Weight (12.35%)

Q4 2024:
--------------------
Best Return: Benchmark (-0.82%)
Best Sharpe: Benchmark (-0.534)
Lowest Vol: Model Portfolio (8.44%)

Q1 2025:
--------------------
Best Return: MVO Portfolio (8.27%)
Best Sharpe: MVO Portfolio (3.257)
Lowest Vol: MVO Portfolio (9.96%)

Q2 2025:
--------------------
Best Return: Model Portfolio (10.70%)
Best Sharpe: MVO Portfolio (2.206)
Lowest Vol: MVO Portfolio (17.95%)

Q3 2025:
--------------------
Best Return: Equal Weight (2.74%)
Best Sharpe: MVO Portfolio (7.344)
Lowest Vol: MVO Portfolio (5.06%)

PORTFOLIO RANKINGS BY QU

### Yearly Comparison

In [27]:
# Main Analysis
print("Yearly Performance Analysis")
print("=" * 50)

# Get yearly periods
yearly_periods = get_yearly_periods(rebalance_dates, data)

# Portfolio definitions
portfolios = {
    'Model Portfolio': model_weights_df,
    'MVO Portfolio': mvo_weights_df,
    'Equal Weight': equal_weights_df,
    'Benchmark': benchmark_df
}

# Calculate yearly performance for each portfolio
yearly_results = {}
weights_df_2 = weights_df.copy()
for portfolio_name, weights_df_2 in portfolios.items():
    yearly_results[portfolio_name] = {}
    
    for period in yearly_periods:
        year = period['year']
        returns = calculate_yearly_portfolio_returns(data, weights_df_2, period, rebalance_dates)
        metrics = calculate_yearly_metrics(returns)
        yearly_results[portfolio_name][year] = metrics

# Create comprehensive results DataFrame
all_metrics = ['Total Return (%)', 'Annualized Return (%)', 'Volatility (%)', 
               'Sharpe Ratio', 'Sortino Ratio', 'Max Drawdown (%)', 
               'Max DD Duration (days)', 'VaR 95% (%)', 'Calmar Ratio', 'Trading Days']

yearly_comparison = {}

for metric in all_metrics:
    yearly_comparison[metric] = pd.DataFrame({
        portfolio: {year: yearly_results[portfolio][year][metric] 
                   for year in yearly_results[portfolio]}
        for portfolio in portfolios.keys()
    })

# Display results
for metric in all_metrics:
    print(f"\n{metric}")
    print("-" * 40)
    if metric in ['Trading Days']:
        print(yearly_comparison[metric].astype(int))
    else:
        print(yearly_comparison[metric].round(3))


Yearly Performance Analysis

Total Return (%)
----------------------------------------
      Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
2024           14.588         11.728        12.287     15.428
2025           13.878         22.116        14.005     12.312

Annualized Return (%)
----------------------------------------
      Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
2024           14.650         11.778        12.339     15.494
2025           26.568         43.652        26.823     23.429

Volatility (%)
----------------------------------------
      Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
2024            9.993         10.626         9.619      9.942
2025           17.623         13.613        16.311     16.973

Sharpe Ratio
----------------------------------------
      Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
2024            1.218          0.913         1.050      1.298
2025            1.311          2.583         1.415     

In [28]:
# Plotting
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Annual Returns (%)', 'Annual Volatility (%)', 
                   'Annual Sharpe Ratio', 'Annual Max Drawdown (%)'),
                   #'Annual Sortino Ratio', 'Annual Calmar Ratio'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
years = list(yearly_comparison['Total Return (%)'].index)

# Plot annual returns
for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=years,
            y=yearly_comparison['Total Return (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=True
        ),
        row=1, col=1
    )

# Plot annual volatility
for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=years,
            y=yearly_comparison['Volatility (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=1, col=2
    )

# Plot annual Sharpe ratio
for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=years,
            y=yearly_comparison['Sharpe Ratio'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=2, col=1
    )

# Plot annual max drawdown
for i, portfolio in enumerate(portfolios.keys()):
    fig.add_trace(
        go.Bar(
            x=years,
            y=yearly_comparison['Max Drawdown (%)'][portfolio],
            name=portfolio,
            marker_color=colors[i],
            showlegend=False
        ),
        row=2, col=2
    )

# # Plot annual Sortino ratio
# for i, portfolio in enumerate(portfolios.keys()):
#     fig.add_trace(
#         go.Scatter(
#             x=years,
#             y=yearly_comparison['Sortino Ratio'][portfolio],
#             mode='lines+markers',
#             name=portfolio,
#             line=dict(color=colors[i]),
#             showlegend=False
#         ),
#         row=3, col=1
#     )

# # Plot annual Calmar ratio
# for i, portfolio in enumerate(portfolios.keys()):
#     fig.add_trace(
#         go.Scatter(
#             x=years,
#             y=yearly_comparison['Calmar Ratio'][portfolio],
#             mode='lines+markers',
#             name=portfolio,
#             line=dict(color=colors[i]),
#             showlegend=False
#         ),
#         row=3, col=2
#     )

# Update layout
fig.update_layout(
    title='Annual Portfolio Performance Comparison',
    height=900,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
fig.update_yaxes(title_text="Return (%)", row=1, col=1)
fig.update_yaxes(title_text="Volatility (%)", row=1, col=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
fig.update_yaxes(title_text="Max Drawdown (%)", row=2, col=2)
# fig.update_yaxes(title_text="Sortino Ratio", row=3, col=1)
# fig.update_yaxes(title_text="Calmar Ratio", row=3, col=2)

fig.show()

# Save the plot to a file
fig.write_html(f'{output_dir}/Others/{model_type}/{train_type}/YoY_performance_{model_type}_run_{run_no}.html')
fig.write_image(f'{output_dir}/Others/{model_type}/{train_type}/YoY_performance_{model_type}_run_{run_no}.png')

In [29]:
# Summary Statistics by Year
print("\n" + "="*60)
print("ANNUAL PERFORMANCE SUMMARY")
print("="*60)

for year in years:
    print(f"\n{year}:")
    print("-" * 20)
    
    # Best performing portfolio this year
    year_returns = yearly_comparison['Total Return (%)'].loc[year]
    best_return = year_returns.idxmax()
    print(f"Best Return: {best_return} ({year_returns[best_return]:.2f}%)")
    
    # Best Sharpe ratio this year
    year_sharpe = yearly_comparison['Sharpe Ratio'].loc[year].dropna()
    if not year_sharpe.empty:
        best_sharpe = year_sharpe.idxmax()
        print(f"Best Sharpe: {best_sharpe} ({year_sharpe[best_sharpe]:.3f})")
    
    # Lowest volatility this year
    year_vol = yearly_comparison['Volatility (%)'].loc[year].dropna()
    if not year_vol.empty:
        lowest_vol = year_vol.idxmin()
        print(f"Lowest Vol: {lowest_vol} ({year_vol[lowest_vol]:.2f}%)")
    
    # Best Calmar ratio this year
    year_calmar = yearly_comparison['Calmar Ratio'].loc[year].dropna()
    if not year_calmar.empty:
        best_calmar = year_calmar.idxmax()
        print(f"Best Calmar: {best_calmar} ({year_calmar[best_calmar]:.3f})")

# Ranking Analysis
print("\n" + "="*60)
print("PORTFOLIO RANKINGS BY YEAR")
print("="*60)

ranking_df = pd.DataFrame(index=years, columns=portfolios.keys())

for year in years:
    # Rank by annual returns (1 = best)
    year_returns = yearly_comparison['Total Return (%)'].loc[year]
    ranks = year_returns.rank(ascending=False, method='min')
    ranking_df.loc[year] = ranks

print("\nRanking by Annual Returns (1=Best, 4=Worst):")
print(ranking_df.astype(int))

# Average ranking
avg_ranking = ranking_df.mean().sort_values()
print(f"\nAverage Ranking Across All Years:")
print("-" * 40)
for portfolio, avg_rank in avg_ranking.items():
    print(f"{portfolio}: {avg_rank:.2f}")

# Win rate analysis
print(f"\nAnnual Win Rate (% of years ranked #1):")
print("-" * 45)
for portfolio in portfolios.keys():
    win_rate = (ranking_df[portfolio] == 1).sum() / len(years) * 100
    print(f"{portfolio}: {win_rate:.1f}%")

# Multi-year consistency analysis
print(f"\nConsistency Analysis:")
print("-" * 25)
for portfolio in portfolios.keys():
    returns_series = yearly_comparison['Total Return (%)'][portfolio].dropna()
    if len(returns_series) > 1:
        consistency = returns_series.std()
        print(f"{portfolio} - Return Std Dev: {consistency:.2f}%")

# Best and worst years
print(f"\nBest and Worst Years:")
print("-" * 25)
for portfolio in portfolios.keys():
    returns_series = yearly_comparison['Total Return (%)'][portfolio].dropna()
    if len(returns_series) > 0:
        best_year = returns_series.idxmax()
        worst_year = returns_series.idxmin()
        print(f"{portfolio}:")
        print(f"  Best: {best_year} ({returns_series[best_year]:.2f}%)")
        print(f"  Worst: {worst_year} ({returns_series[worst_year]:.2f}%)")


ANNUAL PERFORMANCE SUMMARY

2024:
--------------------
Best Return: Benchmark (15.43%)
Best Sharpe: Benchmark (1.298)
Lowest Vol: Equal Weight (9.62%)
Best Calmar: Benchmark (2.385)

2025:
--------------------
Best Return: MVO Portfolio (22.12%)
Best Sharpe: MVO Portfolio (2.583)
Lowest Vol: MVO Portfolio (13.61%)
Best Calmar: MVO Portfolio (5.306)

PORTFOLIO RANKINGS BY YEAR

Ranking by Annual Returns (1=Best, 4=Worst):
      Model Portfolio  MVO Portfolio  Equal Weight  Benchmark
2024                2              4             3          1
2025                3              1             2          4

Average Ranking Across All Years:
----------------------------------------
Model Portfolio: 2.50
MVO Portfolio: 2.50
Equal Weight: 2.50
Benchmark: 2.50

Annual Win Rate (% of years ranked #1):
---------------------------------------------
Model Portfolio: 0.0%
MVO Portfolio: 50.0%
Equal Weight: 0.0%
Benchmark: 50.0%

Consistency Analysis:
-------------------------
Model Portfolio - Re

In [30]:
# --- เริ่มโค้ดสำหรับบันทึก Excel ---
output_excel_path = f"{output_dir}/Others/{model_type}/{train_type}/03_Yearly_{model_type}_{pe_type}_{pre_post}_run_{run_no}.xlsx"

for metric in all_metrics:
    sheet_name = metric.replace(" ", "_")
    df_to_save = yearly_comparison[metric].round(4)
    save_dataframe_to_new_sheet(df_to_save, output_excel_path, sheet_name)


DataFrame saved to sheet 'Total_Return_(%)' in new file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx ✨
DataFrame saved to sheet 'Annualized_Return_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Volatility_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Sharpe_Ratio' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Sortino_Ratio' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Max_Drawdown_(%)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Yearly_tAPE__PreNorm_run_11.xlsx 📄
DataFrame saved to sheet 'Max_DD_Duration_(days)' in existing file: Results/Others/tAPE/01_1_tAPE__PreNorm_temp_1.0/03_Ye

In [31]:
yearly_comparison['Sortino Ratio']

,Model Portfolio,MVO Portfolio,Equal Weight,Benchmark
2024,1.720867,1.176516,1.450152,1.718073
2025,1.691754,2.923963,1.769281,1.488484


In [32]:
os.system('say "All code has finished"')

0